In [1]:
import os
import time
import requests
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

# 🔧 1. 저장 폴더 설정
folder_path = './my_movie_project_1'
os.makedirs(folder_path, exist_ok=True)

# 🔐 2. API 키
tmdb_api_key = 'YOUR_TMDB_API_KEY'     # ← 여기에 본인의 TMDB API 키 입력
omdb_api_key = 'YOUR_OMDB_API_KEY'     # ← 여기에 본인의 OMDb API 키 입력

# 🔗 3. API URL
tmdb_discover_url = 'https://api.themoviedb.org/3/discover/movie'
tmdb_detail_url = 'https://api.themoviedb.org/3/movie/{}'
tmdb_keywords_url = 'https://api.themoviedb.org/3/movie/{}/keywords'
omdb_url = 'http://www.omdbapi.com/'

headers = {'Accept': 'application/json'}
language = 'ko-KR'  # 한글 정보

# 🔁 4. 요청 재시도 설정 함수
def requests_retry_session(retries=3, backoff_factor=1.0, status_forcelist=(500, 502, 504)):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    return session

# 🎬 5. 연도별 영화 수집
all_movies = []

for year in range(2015, 2026):
    year_movies = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        params = {
            'api_key': tmdb_api_key,
            'language': language,
            'sort_by': 'popularity.desc',
            'primary_release_year': year,
            'page': page
        }

        try:
            response = requests_retry_session().get(tmdb_discover_url, headers=headers, params=params, timeout=10)
            data = response.json()
        except Exception as e:
            print(f"❌ TMDB 요청 실패 (연도: {year}, 페이지: {page}) → {e}")
            page += 1
            continue

        if page == 1:
            total_pages = data.get('total_pages', 1)

        for movie in data.get('results', []):
            tmdb_id = movie.get('id')
            title = movie.get('title')
            release_date = movie.get('release_date')
            vote_average = movie.get('vote_average')
            media_type = "movie"

            # ▶ TMDB 상세 정보
            try:
                detail_resp = requests_retry_session().get(
                    tmdb_detail_url.format(tmdb_id),
                    headers=headers,
                    params={'api_key': tmdb_api_key, 'language': language},
                    timeout=10
                )
                detail = detail_resp.json()
            except Exception as e:
                print(f"⚠️ TMDB 상세 실패 - {tmdb_id}: {e}")
                continue

            imdb_id = detail.get('imdb_id')
            runtime = detail.get('runtime')
            genres = ", ".join([g['name'] for g in detail.get('genres', [])])
            production_companies = detail.get('production_companies', [])
            production_company = production_companies[0]['name'] if production_companies else None

            # ▶ TMDB 키워드
            try:
                keyword_resp = requests_retry_session().get(
                    tmdb_keywords_url.format(tmdb_id),
                    headers=headers,
                    params={'api_key': tmdb_api_key},
                    timeout=10
                )
                keyword_data = keyword_resp.json()
                tags = ", ".join([kw['name'] for kw in keyword_data.get('keywords', [])])
            except:
                tags = None

            # ▶ OMDb 정보
            director = cast = country = language_omdb = plot = imdb_rating = imdb_votes = None
            if imdb_id:
                try:
                    omdb_params = {'apikey': omdb_api_key, 'i': imdb_id}
                    omdb_resp = requests_retry_session().get(omdb_url, params=omdb_params, timeout=10)
                    omdb_data = omdb_resp.json()

                    if omdb_data.get('Response') == 'True':
                        director = omdb_data.get('Director')
                        cast = omdb_data.get('Actors')
                        country = omdb_data.get('Country')
                        language_omdb = omdb_data.get('Language')
                        plot = omdb_data.get('Plot')
                        imdb_rating = omdb_data.get('imdbRating')
                        imdb_votes = omdb_data.get('imdbVotes')
                    else:
                        print(f"⚠️ OMDb 응답 없음 - {imdb_id}")
                except:
                    print(f"⚠️ OMDb 요청 실패 - {imdb_id}")

            movie_data = {
                'title': title,
                'release_date': release_date,
                'type': media_type,
                'runtime': runtime,
                'genre': genres,
                'director': director,
                'cast': cast,
                'country': country,
                'language': language_omdb,
                'production_company': production_company,
                'synopsis': plot,
                'tags': tags,
                'imdb_rating': imdb_rating,
                'imdb_review_count': imdb_votes,
                'imdb_id': imdb_id
            }

            year_movies.append(movie_data)
            time.sleep(0.3)

        print(f"[{year}] 페이지 {page}/{total_pages} 수집 완료")
        page += 1

    # ✅ 연도별 체크포인트 저장
    if year_movies:
        df = pd.DataFrame(year_movies)
        df.to_csv(f"{folder_path}/tmdb_imdb_checkpoint_{year}.csv", index=False)
        all_movies.extend(year_movies)
        print(f"✅ 체크포인트 저장 완료: {year}")
    else:
        print(f"⚠️ 연도 {year}에 저장할 데이터가 없습니다.")

# 🗃️ 전체 결과 저장
if all_movies:
    final_df = pd.DataFrame(all_movies)
    final_df.to_csv(f"{folder_path}/tmdb_imdb_combined_2015_2025.csv", index=False)
    print("🎉 전체 영화 데이터 저장 완료!")
else:
    print("❌ 수집된 데이터가 없습니다.")


[2015] 페이지 1/1 수집 완료
⚠️ 연도 2015에 저장할 데이터가 없습니다.
[2016] 페이지 1/1 수집 완료
⚠️ 연도 2016에 저장할 데이터가 없습니다.
[2017] 페이지 1/1 수집 완료
⚠️ 연도 2017에 저장할 데이터가 없습니다.
[2018] 페이지 1/1 수집 완료
⚠️ 연도 2018에 저장할 데이터가 없습니다.
[2019] 페이지 1/1 수집 완료
⚠️ 연도 2019에 저장할 데이터가 없습니다.
[2020] 페이지 1/1 수집 완료
⚠️ 연도 2020에 저장할 데이터가 없습니다.
[2021] 페이지 1/1 수집 완료
⚠️ 연도 2021에 저장할 데이터가 없습니다.
[2022] 페이지 1/1 수집 완료
⚠️ 연도 2022에 저장할 데이터가 없습니다.
[2023] 페이지 1/1 수집 완료
⚠️ 연도 2023에 저장할 데이터가 없습니다.
[2024] 페이지 1/1 수집 완료
⚠️ 연도 2024에 저장할 데이터가 없습니다.
[2025] 페이지 1/1 수집 완료
⚠️ 연도 2025에 저장할 데이터가 없습니다.
❌ 수집된 데이터가 없습니다.


In [2]:
print(response.status_code)
print(data.keys())
print(data.get('results'))
print(data.get('total_results'))

401
dict_keys(['status_code', 'status_message', 'success'])
None
None


In [3]:
print(params)

{'api_key': 'YOUR_TMDB_API_KEY', 'language': 'ko-KR', 'sort_by': 'popularity.desc', 'primary_release_year': 2025, 'page': 1}


In [4]:
if 'status_message' in data:
    print("TMDb Error:", data['status_message'])

TMDb Error: Invalid API key: You must be granted a valid key.


In [5]:
year = 2023
page = 1
# ... 요청 후 print(response.status_code, len(data.get('results', [])))

In [7]:
response = requests_retry_session().get(tmdb_discover_url, headers=headers, params=params, timeout=10)
print(f"[DEBUG] {year=} {page=} status={response.status_code}")
data = response.json()
print(data.get('results')[:3])  # 상위 3개 결과 미리 보기

[DEBUG] year=2023 page=1 status=401


TypeError: 'NoneType' object is not subscriptable

In [8]:
response = requests_retry_session().get(tmdb_discover_url, headers=headers, params=params, timeout=10)
print(f"[DEBUG] {year=} {page=} status={response.status_code}")

try:
    data = response.json()
except Exception as e:
    print(f"[ERROR] JSON 디코딩 실패: {e}")
    data = {}

results = data.get('results')
if results:
    print(results[:3])  # 결과 일부 출력
else:
    print(f"[WARNING] No results in data: {data}")

[DEBUG] year=2023 page=1 status=401
[WARNING] No results in data: {'status_code': 7, 'status_message': 'Invalid API key: You must be granted a valid key.', 'success': False}
